# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_to_load = "output_data/cities.csv"
cities_df = pd.read_csv(file_to_load)
print(cities_df.shape)
cities_df.head(3)

(556, 11)


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Min Temp,Max Temp,Wind Speed
0,Belaya Gora,33,RU,1603459267,92,68.53,146.42,5.07,5.07,5.07,3.67
1,Luwuk,16,ID,1603459267,78,-0.95,122.79,78.93,78.93,78.93,0.47
2,Vaini,75,TO,1603459267,88,-21.20,-175.20,71.60,71.60,71.60,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Using Lat & Lng as locations 
locations = cities_df[["Lat", "Lng"]].astype(float)
locations.head(2)

,Lat,Lng
0,68.53,146.42
1,-0.95,122.79


In [5]:
#Defining Humudity as the weight
humidity = cities_df["Humidity"].astype(float)
humidity.head(2)

0    92.0
1    78.0
Name: Humidity, dtype: float64

In [6]:
# Creating a Heatmap layer
heat_fig = gmaps.figure(center=(46.0, -5.0),zoom_level=2) # 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 5)
heat_fig.add_layer(heat_layer)
heat_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#Narrowing down cities by (i) max_temp<80 & min_temp>70; (ii) Wind Speed < 10; (iii) Cloudiness == 0
vacation_places = cities_df.loc[(cities_df["Min Temp"]>70)&(cities_df["Max Temp"]<80)
                                &(cities_df["Wind Speed"]<10)&(cities_df["Cloudiness"]==0)]
vacation_places = vacation_places.dropna()
print(vacation_places.shape)
vacation_places

(11, 11)


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Min Temp,Max Temp,Wind Speed
200,East London,0,ZA,1603459330,78,-33.02,27.91,75.20,75.20,75.20,6.93
210,Khoy,0,IR,1603459130,25,38.55,44.95,70.56,70.56,70.56,6.04
286,La Reforma,0,MX,1603459355,91,25.08,-108.05,73.58,71.01,77.00,3.00
302,Koróni,0,GR,1603459360,44,36.80,21.95,75.09,75.00,75.20,8.05
333,Kumluca,0,TR,1603459368,39,36.37,30.29,79.45,79.45,79.45,5.10
339,Diego de Almagro,0,CL,1603459369,27,-26.37,-70.05,70.50,70.50,70.50,4.03
397,Bani Walid,0,LY,1603459384,26,31.76,13.99,79.30,79.30,79.30,5.64
424,Parbhani,0,IN,1603459392,59,19.27,76.78,79.36,79.36,79.36,4.45
425,Kondagaon,0,IN,1603459392,73,19.60,81.67,71.46,71.46,71.46,5.35
472,San Juan,0,AR,1603459405,42,-31.54,-68.54,74.80,74.80,74.80,5.79


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#Getting the Lat & Lng of the vacation locations
vaca_locations = vacation_places[["Lat", "Lng"]].astype(float)
print(vaca_locations.shape)
vaca_locations.head(2)

(11, 2)


,Lat,Lng
200,-33.02,27.91
210,38.55,44.95


In [10]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [11]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(vaca_locations)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [13]:
#Adding the columns of Hotel Name, Address and Rating to the Vacation places df
vacation_places["Hotel Name"] =""
vacation_places["Hotel Address"] =""
vacation_places["Hotel Rating"] =""
vacation_places = vacation_places.reset_index()
print(vacation_places.shape)
vacation_places.head(3)

(11, 16)


,level_0,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Min Temp,Max Temp,Wind Speed,Hotel Name,Hotel Address,Hotel Rating
0,0,200,East London,0,ZA,1603459330,78,-33.02,27.91,75.20,75.20,75.20,6.93,,,
1,1,210,Khoy,0,IR,1603459130,25,38.55,44.95,70.56,70.56,70.56,6.04,,,
2,2,286,La Reforma,0,MX,1603459355,91,25.08,-108.05,73.58,71.01,77.00,3.00,,,


In [17]:
def getHotel(lat,lng):
    params = {
        "keyword": "Hotel",
        "types": "lodging",
        "radius": 50000,
        "key": g_key,
        "location": f"{lat}, {lng}"
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params).json()
    pprint(response)    
    return response


In [18]:
results = []

for i, row in vacation_places.iterrows():
    function_response = getHotel(row["Lat"], row["Lng"])
    results.append(function_response)

{'html_attributions': [],
 'next_page_token': 'CrQCIwEAAMVrkKiu06lsp4HkltEqKQImjmIsOcgA6RvwmCemsmDT9Mmcv4ga1VS_JKtRCeT_R-_OIoyrDOAv5B-7aD5B6T9RL2swqovUFYKu9wj14LYJ1sS7IFhN15I63F5I19yj4ofAMUsfp2ujQYIiW7IXxTAhYz7pV0H9biJIuVwJwyDcU_1xD5chzem7oJmFxEyvtqLHxKFknx6jiTdNLirL8Q6z8CqPjxf_upa_Yncp90XjzNfuS6_sdvhCug1DuX15c-GonF_Fq_ixYJu5E_uxB0vYxebqIJfcVIwFeB9g_WR4MLWB3otJ7BTWw9Wkyr6a2AWUtBkI5heFPWB7Tk9uh7MAx_nZW8YiBCQsqt__L8rJ6KJX9eh6o0W52nVHLFBCApAgUG0LSmC9PxNDaZ3c6gQSEOJ4vfzug6xXiTiSxNzCQfUaFGEwS0oCTtUgly-_a4aJIDWhA5s-',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -32.8390228, 'lng': 28.0776644},
                           'viewport': {'northeast': {'lat': -32.83749242010727,
                                                      'lng': 28.07891472989272},
                                        'southwest': {'lat': -32.84019207989272,
                                                      'lng': 28.07621507010728}}},
              'icon': 'https

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 38.1919338, 'lng': 44.7663805},
                           'viewport': {'northeast': {'lat': 38.19333307989272,
                                                      'lng': 44.76773247989271},
                                        'southwest': {'lat': 38.19063342010728,
                                                      'lng': 44.76503282010727}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Shahin hotel',
              'place_id': 'ChIJjRk8AoSvEUARn3ZECR4lCRI',
              'plus_code': {'compound_code': '5QR8+QH Salmas, West Azerbaijan '
                                             'Province, Iran',
                            'global_code': '8HC65QR8+QH'},
              'rating': 5,
              'reference': 'ChIJjRk8AoSvEUARn3ZECR4lCRI',
              'scope': 'GOOGLE',
        

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 25.4832469, 'lng': -107.9215148},
                           'viewport': {'northeast': {'lat': 25.48459212989272,
                                                      'lng': -107.9201068701073},
                                        'southwest': {'lat': 25.48189247010728,
                                                      'lng': -107.9228065298927}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'La Cuarteria Hotel Boutique',
              'photos': [{'height': 2268,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/110251811440476302299">A '
                                                'Google User</a>'],
                          'photo_reference': 'CmRZAAAAVXXrN1pCn7cD0O2MjxCfgD9ERGtcv

{'html_attributions': [],
 'next_page_token': 'CrQCIwEAAI6xJ4Avaw2SlfBI0wNfJmJSnfuE6VKuecKj6W_dn9C3qorhD73nTJNxlJYU92x9etDSsLGPXNXgNYq9hCMu3SpSmckMyyyxdW-kSUrSYYIRN_p5pW3NzRJJJrP0lVh5KfLZzrCacIUQyqevGgxWgVBWL763nlpSxxS7mGWrWc1TRpyczXWRtTYs48CvOxbu1uHEBovsylnWkGkSAtFGVjX5n2aEyzKVVVQCk-4dw1iulZrU7br2KFljGsNnCbHUItSsCPk9yy2oqPEhGgnjM3zh8cDFjiHrktXIj87fVZhvwZCNYZXkWJmVuYp8PbnX2FzWsYjx9hhw7SExMHQ-jAIrCOeUZyO7-LQmYYh-6wkoNWdS_EQu9DqLXcv55ugOpSxKfnNVNz7siuONv3hZABgSEIzn2GgNz2WND8BplGHlq90aFAmfsv4vCHzqpC0EkUvl1cFuXw4D',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 36.479751, 'lng': 22.4006468},
                           'viewport': {'northeast': {'lat': 36.48124527989273,
                                                      'lng': 22.40207862989272},
                                        'southwest': {'lat': 36.47854562010728,
                                                      'lng': 22.39937897010728}}},
              'icon': 'https://m

{'html_attributions': [],
 'next_page_token': 'CrQCIwEAAGXY8QlJcI6Idz0oL-m92kM1rBpZoO5LTHGKgu5OPmQzJjLStAK-_zYczhRA79HdeXOGt7iJ8wH4BtsKf9glN3q96iH9iJxaAxg8YzOB-xH49AKbdsRo5RYuvasKVdSHo3KqyK6aI0_ovXw4t5T8FnOy1IOGgSrNNkpTlhf3T0OZAib-jajq5hMISGBT1QcMUJKAnoDilZ01ghdCHhbrmXVUukgsHdDb0I_aDYRYfYaRzjDY5ZmHYU98HYvf9vBH9oZDihMTsbOwFrGXWLDG5vsgZVyWw6crEaFCLK36EAM_oNeD6QX8iuuvItje-5qIAGKq47jZVOSNBvxdzZ6WCEhC6EpUHI39nhhhqsA9pF01YOUVlSnhgQ5S-th3U-zIyBrcIFlG-80lq8ePLGEsI48SEL_fw9zXYBOPtjA5mCIknaUaFNLG2isNDYfvmeL82-34CVX8tKtQ',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 36.509325, 'lng': 30.537292},
                           'viewport': {'northeast': {'lat': 36.51064872989272,
                                                      'lng': 30.53865212989272},
                                        'southwest': {'lat': 36.50794907010727,
                                                      'lng': 30.53595247010728}}},
              'icon': 'https://ma

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -26.2474347, 'lng': -69.6247901},
                           'viewport': {'northeast': {'lat': -26.24608487010728,
                                                      'lng': -69.62344027010728},
                                        'southwest': {'lat': -26.24878452989272,
                                                      'lng': -69.62613992989273}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Cabañas El Salvador, El Salvador , Chile',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 1728,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/105199465036849640226">A '
                                                'Google User</a>'],
                 

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 31.755946, 'lng': 14.014865},
                           'viewport': {'northeast': {'lat': 31.75723782989272,
                                                      'lng': 14.01613377989272},
                                        'southwest': {'lat': 31.75453817010728,
                                                      'lng': 14.01343412010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'فندق الزيتونة',
              'photos': [{'height': 720,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/102829558396658498203">محمد '
                                                'الحطماني</a>'],
                          'photo_reference': 'CmRaAAAAlU-kJE-Lnq8axh2kLOK_QPxiDsd2EeLBDTIxiJvxJVuoQVEWjY

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 19.205, 'lng': 81.6986},
                           'viewport': {'northeast': {'lat': 19.20624957989272,
                                                      'lng': 81.70015212989273},
                                        'southwest': {'lat': 19.20354992010727,
                                                      'lng': 81.6974524701073}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Dandami Luxury Resort',
              'photos': [{'height': 1371,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/111091063567123873366">Saurabh '
                                                'Yadav</a>'],
                          'photo_reference': 'CmRaAAAAATId8Euc4h_DSGFfrfwUWLb4rmDcSRcWSJNMxSAqokVcS0i

{'html_attributions': [],
 'next_page_token': 'CrQCIwEAAJKHJ2PSTAKBGHw045sBWVzv31JWh_IKzICU_N3HPidWnHbc2b-hzbEbQjgyUbxeA4uwWOz0ee-xu03F8mT9eI6yph29g2mK-qAT0ztwGaL4hZptQZ-qGU_2UIudh4RdLi4OdxKjckDGIGqlubxw0MH90UzSlmCxobwY2nCGSnKphPMJB_vu1nunORhtfwDtgHKSa5Lp8QJDMVrShYbDw2UTFimUH6eHyOcrXcxqKtioEMJFACfQhRYqRXAws9CncHfNLwns20iu58SWFilQu87pBleSF3FwB-6VNQd7OBf32hwbWom9yOInguA6D7znuLTRXI2MYUM1yx4OGDMYMjx51D406zoPoEH0ISysY0b9VSJiA6oLEZrforI3L6Rfa_U4ZN3-gOArSNSAcysFmxoSEH9_G7qGdMQjBnArlBLAvCEaFPH00_kwMw8uL2uK5b5_SJNlMzOM',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 38.8635168, 'lng': 0.0190487},
                           'viewport': {'northeast': {'lat': 38.86487132989271,
                                                      'lng': 0.02016612989272221},
                                        'southwest': {'lat': 38.86217167010727,
                                                      'lng': 0.01746647010727778}}},
              'icon': 'https

In [19]:
print(len(results))
print(results[0]["results"][0]["name"])
print(results[0]["results"][0]["vicinity"])
print(results[0]["results"][0]["rating"])

11
Inkwenkwezi Private Game Reserve
Schafli Rd, Chintsa West, East London
4.4


In [20]:
for i, row in vacation_places.iterrows():
    try:
        vacation_places.loc[i,"Hotel Name"] = results[i]["results"][0]["name"]
        vacation_places.loc[i, "Hotel Address"] = results[i]["results"][0]["vicinity"]
        vacation_places.loc[i, "Hotel Rating"]= results[i]["results"][0]["rating"]
    except:
        print(f"Missing result for {row['City']}")

In [21]:
vacation_places

,level_0,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Min Temp,Max Temp,Wind Speed,Hotel Name,Hotel Address,Hotel Rating
0,0,200,East London,0,ZA,1603459330,78,-33.02,27.91,75.20,75.20,75.20,6.93,Inkwenkwezi Private Game Reserve,"Schafli Rd, Chintsa West, East London",4.4
1,1,210,Khoy,0,IR,1603459130,25,38.55,44.95,70.56,70.56,70.56,6.04,Shahin hotel,"Salmas, ‫روستای طالقانی،",5
2,2,286,La Reforma,0,MX,1603459355,91,25.08,-108.05,73.58,71.01,77.00,3.00,La Cuarteria Hotel Boutique,"Francisco I. Madero 63, Centro, Mocorito",4.6
3,3,302,Koróni,0,GR,1603459360,44,36.80,21.95,75.09,75.00,75.20,8.05,Kyrimai Hotel,"Ν. Λακωνίας, Gerolimenas",4.6
4,4,333,Kumluca,0,TR,1603459368,39,36.37,30.29,79.45,79.45,79.45,5.10,Rixos Premium Tekirova,"Şehit Er Hasan Yılmaz Cad. No:20, Kemer",4.6
5,5,339,Diego de Almagro,0,CL,1603459369,27,-26.37,-70.05,70.50,70.50,70.50,4.03,"Cabañas El Salvador, El Salvador , Chile","Av. Potrerillos Sur 2306, El Salvador, Diego d...",5
6,6,397,Bani Walid,0,LY,1603459384,26,31.76,13.99,79.30,79.30,79.30,5.64,فندق الزيتونة,Bani Waled,4
7,7,424,Parbhani,0,IN,1603459392,59,19.27,76.78,79.36,79.36,79.36,4.45,"The Fern Residency, Parbhani","Plot No P-12, Basmat Rd, Maharashtra Industria...",4.2
8,8,425,Kondagaon,0,IN,1603459392,73,19.60,81.67,71.46,71.46,71.46,5.35,Dandami Luxury Resort,Tiratha,4
9,9,472,San Juan,0,AR,1603459405,42,-31.54,-68.54,74.80,74.80,74.80,5.79,Oasis Cordillerano Village,"J5400BHL, Cereceto Este 546, San Juan",4.5


In [22]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_places.iterrows()]
locations = vaca_locations[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
hotel_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
heat_fig.add_layer(hotel_markers)
# Display Map
heat_fig

Figure(layout=FigureLayout(height='420px'))